In [ ]:
import schedule
from datetime import datetime, timedelta

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import requests, json

In [ ]:
#lien qui montre le nombre de vélos par borne en temps réél
import pandas as pd
import requests, json
link = ('https://data.rennesmetropole.fr/api/records/1.0/search/?dataset=etat-des-stations-le-velo-star-en-temps-reel&q=&rows=200&facet=nom&facet=etat&facet=nombreemplacementsactuels&facet=nombreemplacementsdisponibles&facet=nombrevelosdisponibles')
a = requests.get(link).json()
a


In [3]:
df = pd.json_normalize(data = a , record_path = ["records"])
df.shape

(57, 13)

In [ ]:
type('fields.lastupdate')


str

In [4]:
'''diviser colonnes date, créer colonnes, convertir en datetime, modif dict'''
import numpy as np 
from datetime import datetime, timedelta
df["fields.lastupdate"] = pd.to_datetime(df["fields.lastupdate"])
df['Dates'] = pd.to_datetime(df["fields.lastupdate"]).dt.date
df['Time'] = (pd.to_datetime(df["fields.lastupdate"]).dt.time) 
df['Times']= pd.to_datetime(df['Time'].astype(str)) + pd.DateOffset(hours=2)
df['Dates'] = pd.to_datetime(df['Dates'])
df.head()
df[df["fields.nom"]== "TNB"]

,datasetid,recordid,record_timestamp,fields.nombrevelosdisponibles,fields.nombreemplacementsdisponibles,fields.nom,fields.nombreemplacementsactuels,fields.idstation,fields.etat,fields.coordonnees,fields.lastupdate,geometry.type,geometry.coordinates,Dates,Time,Times
36,etat-des-stations-le-velo-star-en-temps-reel,7e983dd5384bc61ab8d85cc20933235be20e7941,2022-06-29T05:45:00Z,5,23,TNB,28,5512,En fonctionnement,"[48.107748, -1.673711]",2022-06-29 07:44:07+00:00,Point,"[-1.673711, 48.107748]",2022-06-29,07:44:07,2022-06-29 09:44:07


In [10]:
#création de la map et ajout des bornes de vélos 
import folium
velos_map = folium.Map(location=[48.107748, -1.673711]	, tiles="OpenStreetMap", zoom_start=12, control_scale= True)
# add marker one by one on the map
for index, row in df.iterrows():
   folium.Marker(
      location=row["fields.coordonnees"],
      popup= 'Station : ' + str (row["fields.nom"]) + '<br>' + 'Vélos disponibles : ' + str (row["fields.nombrevelosdisponibles"]) ,
      # icon = folium.Icon(icon=str (df_extract_samedi24.iloc[i]['bike_stands']), prefix="fa")
      icon=folium.Icon(icon="bicycle", prefix="fa" )
   ).add_to(velos_map)
velos_map

à partir d'ici je crée une autre map avec des clusters de vélos 

In [11]:
velos_map1 = folium.Map(location=[48.121074, -1.70428]	, tiles="OpenStreetMap", zoom_start=12)


In [12]:
#pour afficher combien d'état il y a par station vélo
df["fields.etat"].unique()

array(['En fonctionnement', 'En panne'], dtype=object)

In [13]:
#création des clusters 
mask1 = df.loc[(df['fields.etat']=='En fonctionnement') & (df['fields.nombrevelosdisponibles'] > 0)] #stations ouvertes avec velos dispos
mask2 = df.loc[(df['fields.etat']=='En fonctionnement') & (df['fields.nombrevelosdisponibles'] == 0)] #stations ouvertes 0 velos dispos
mask3 = df.loc[df['fields.etat'] == 'En panne'] #stations fermées

In [14]:
from folium.plugins import MarkerCluster
import sys
sys.setrecursionlimit(1500)
# add marker one by group on the map
#groupe statio ouverts + velos dispos
open_group1 = folium.FeatureGroup(name="Vélos disponibles")
cluster1 = MarkerCluster().add_to(open_group1)
for index, row in mask1.iterrows():
  folium.Marker(location = row["fields.coordonnees"],
                popup='<strong>'+"Nom de Station:"+'</strong>' + '<br>'+str (row["fields.nom"]) + '<br>' + '<strong>'+"Vélos disponibles:"+'</strong>'+'<br>'+str (row["fields.nombrevelosdisponibles"]),
                icon = folium.Icon(icon ='bicycle' , prefix="fa", color='green')).add_to(cluster1)

open_group2 = folium.FeatureGroup(name="Vélos non disponibles")
cluster2 = MarkerCluster().add_to(open_group2)
for index, row in mask2.iterrows():
  folium.Marker(location = row["fields.coordonnees"], 
                popup='<strong>'+"Nom de Station:"+'</strong>' + '<br>'+str (row["fields.nom"]) + '<br>' + '<strong>'+"Vélos disponibles:"+'</strong>'+'<br>'+str (row["fields.nombrevelosdisponibles"]),
                icon = folium.Icon(icon ='bicycle' , prefix="fa", color='red')).add_to(cluster2)

closed_stations = folium.FeatureGroup(name="Stations fermées")
cluster3 = MarkerCluster().add_to(closed_stations)
for index, row in mask3.iterrows():
  folium.Marker(location = row["fields.coordonnees"], 
                popup='<strong>'+"Nom de Station:"+'</strong>' + '<br>'+str (row["fields.nom"]) + '<br>' + '<strong>'+"Vélos disponibles:"+'</strong>'+'<br>'+str (row["fields.nombrevelosdisponibles"]),
                icon = folium.Icon(icon ='bicycle' , prefix="fa", color='black')).add_to(cluster3)

open_group1.add_to(velos_map1)
open_group2.add_to(velos_map1)
closed_stations.add_to(velos_map1)

In [ ]:
velos_map1


In [16]:
#calcul du nombre moyen de vélo par borne
df["fields.nombreemplacementsactuels"].mean()

24.36842105263158

In [ ]:
velos_map1.save(outfile='map_velos.html')

In [17]:
#plot contenant le nombre de vélos pour chaque borne
import plotly.express as px
fig1 = px.bar(df, x='fields.nom', y='fields.nombreemplacementsactuels', color ='fields.nombreemplacementsactuels', title="nombre de vélos par borne", labels={'fields.nombreemplacementsactuels': "nombre de vélos", "fields.nom": "nom de la station"})
fig1.update_layout(barmode='group')
fig1.show()

In [ ]:
fig1.write_html("velos.html")